##### Copyright 2018 The TensorFlow Probability Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Sobre os formatos do TensorFlow Distributions

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/probability/examples/Understanding_TensorFlow_Distributions_Shapes"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/probability/examples/Understanding_TensorFlow_Distributions_Shapes.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/probability/examples/Understanding_TensorFlow_Distributions_Shapes.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/probability/examples/Understanding_TensorFlow_Distributions_Shapes.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

In [ ]:
import collections

import tensorflow as tf
tf.compat.v2.enable_v2_behavior()

import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

## Fundamentos

Existem três conceitos importantes associados aos formatos do TensorFlow Distributions:

- O *formato do evento* descreve o formato de uma única obtenção de dado da distribuição; a obtenção pode ser dependente entre as dimensões. Para distribuições escalares, o formato de evento é `[]`. Para uma distribuição normal multivariada de 5 dimensões, o formato de evento é `[5]`.
- O *formato de lote* descreve obtenções de dados independentes e não identicamente distribuídas, também conhecidas como "lote" de distribuições.
- O *formato de amostra* descreve obtenções independentes e identicamente distribuídas de lotes da família de distribuições.

O formato de evento e o formato de lote são propriedades de um objeto `Distribution`, em que o formato de amostra está associado a uma chamada específica a `sample` ou `log_prob`.

A finalidade deste notebook é ilustrar esses conceitos por meio de exemplos, então, caso isso não fique óbvio imediatamente, não se preocupe.

Para outra visão conceitual desses conceitos, confira [esta postagem de blog](https://ericmjl.github.io/blog/2019/5/29/reasoning-about-shapes-and-probability-distributions/).

### Observação sobre o TensorFlow Eager:

Este notebook inteiro foi escrito usando-se [TensorFlow Eager](https://research.googleblog.com/2017/10/eager-execution-imperative-define-by.html). Nenhum dos conceitos apresentados *dependem* da execução eager, embora, com ela, os formatos de lote e evento da distribuição sejam avaliados (e, portanto, conhecidos) quando o objeto `Distribution` é criado no Python, enquanto, no modo grafo (que não é o modo eager), é possível definir distribuições cujos formatos de evento e lote sejam indeterminados até a execução do grafo.

## Distribuições escalares

Conforme indicamos acima, um objeto `Distribution` tem formatos de evento e lote definidos. Vamos começar com um utilitário para descrever distribuições:

In [ ]:
def describe_distributions(distributions):
  print('\n'.join([str(d) for d in distributions]))

Nesta seção, vamos explorar distribuições *escalares*, aquelas com um formato de evento igual a `[]`. Um exemplo típico é a distribuição de Poisson, especificada por uma `rate` (taxa):

In [ ]:
poisson_distributions = [
    tfd.Poisson(rate=1., name='One Poisson Scalar Batch'),
    tfd.Poisson(rate=[1., 10., 100.], name='Three Poissons'),
    tfd.Poisson(rate=[[1., 10., 100.,], [2., 20., 200.]],
                name='Two-by-Three Poissons'),
    tfd.Poisson(rate=[1.], name='One Poisson Vector Batch'),
    tfd.Poisson(rate=[[1.]], name='One Poisson Expanded Batch')
]

describe_distributions(poisson_distributions)

tfp.distributions.Poisson("One_Poisson_Scalar_Batch", batch_shape=[], event_shape=[], dtype=float32)
tfp.distributions.Poisson("Three_Poissons", batch_shape=[3], event_shape=[], dtype=float32)
tfp.distributions.Poisson("Two_by_Three_Poissons", batch_shape=[2, 3], event_shape=[], dtype=float32)
tfp.distributions.Poisson("One_Poisson_Vector_Batch", batch_shape=[1], event_shape=[], dtype=float32)
tfp.distributions.Poisson("One_Poisson_Expanded_Batch", batch_shape=[1, 1], event_shape=[], dtype=float32)


A distribuição de Poisson é uma distribuição escalar, então seu formato de evento sempre é igual a `[]`. Se especificarmos mais taxas, elas aparecerão no formato de lote. O par final de exemplos é interessante: há somente uma única taxa, mas, como essa taxa é incorporada a um array NumPy com formato não vazio, esse formato se torna o formato de lote.

A distribuição normal padrão também é um escalar. Seu formato de evento é `[]`, igual à distribuição de Poisson, mas faremos experimentos com ela para ver nosso primeiro exemplo de *broadcasting*. A distribuição normal é especificada usando os parâmetros `loc` e `scale`:

In [ ]:
normal_distributions = [
    tfd.Normal(loc=0., scale=1., name='Standard'),
    tfd.Normal(loc=[0.], scale=1., name='Standard Vector Batch'),
    tfd.Normal(loc=[0., 1., 2., 3.], scale=1., name='Different Locs'),
    tfd.Normal(loc=[0., 1., 2., 3.], scale=[[1.], [5.]],
               name='Broadcasting Scale')
]

describe_distributions(normal_distributions)

tfp.distributions.Normal("Standard", batch_shape=[], event_shape=[], dtype=float32)
tfp.distributions.Normal("Standard_Vector_Batch", batch_shape=[1], event_shape=[], dtype=float32)
tfp.distributions.Normal("Different_Locs", batch_shape=[4], event_shape=[], dtype=float32)
tfp.distributions.Normal("Broadcasting_Scale", batch_shape=[2, 4], event_shape=[], dtype=float32)


O exemplo interessante acima é a distribuição `Broadcasting Scale`. O parâmetro `loc` tem formato `[4]`, e o parâmetro `scale` tem formato `[2, 1]`. Usando as [regras de broadcasting do NumPy](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html), o formato de lote é `[2, 4]`. Uma forma equivalente (mas menos elegante e não recomendada) de definir a distribuição `"Broadcasting Scale"` seria:

In [ ]:
describe_distributions(
    [tfd.Normal(loc=[[0., 1., 2., 3], [0., 1., 2., 3.]],
                scale=[[1., 1., 1., 1.], [5., 5., 5., 5.]])])

tfp.distributions.Normal("Normal", batch_shape=[2, 4], event_shape=[], dtype=float32)


Podemos ver por que a notação de broadcasting é útil, embora também seja uma fonte de dor de cabeça e bugs.

### Amostragem de distribuições escalares

Existem duas ações principais que podemos fazer com as distribuições: podemos amostrá-las (`sample`) e podemos computar as log-probabilidades (`log_prob`s). Vamos explorar a amostragem primeiro. A regra básica é que, quando fazemos a amostragem de uma distribuição, o tensor resultante tem formato `[sample_shape, batch_shape, event_shape]`, em que `batch_shape` e `event_shape` são fornecidos pelo objeto `Distribution`, e `sample_shape` é fornecido pela chamada a `sample`. Para distribuições escalares, `event_shape = []`, então o tensor retornado pela amostra terá formato `[sample_shape, batch_shape]`. Vamos testar:

In [ ]:
def describe_sample_tensor_shape(sample_shape, distribution):
    print('Sample shape:', sample_shape)
    print('Returned sample tensor shape:',
          distribution.sample(sample_shape).shape)

def describe_sample_tensor_shapes(distributions, sample_shapes):
    started = False
    for distribution in distributions:
      print(distribution)
      for sample_shape in sample_shapes:
        describe_sample_tensor_shape(sample_shape, distribution)
      print()

sample_shapes = [1, 2, [1, 5], [3, 4, 5]]
describe_sample_tensor_shapes(poisson_distributions, sample_shapes)

tfp.distributions.Poisson("One_Poisson_Scalar_Batch", batch_shape=[], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1,)
Sample shape: 2
Returned sample tensor shape: (2,)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5)

tfp.distributions.Poisson("Three_Poissons", batch_shape=[3], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 3)
Sample shape: 2
Returned sample tensor shape: (2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 3)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 3)

tfp.distributions.Poisson("Two_by_Three_Poissons", batch_shape=[2, 3], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 2, 3)
Sample shape: 2
Returned sample tensor shape: (2, 2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 2, 3)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 2, 3)

In [ ]:
describe_sample_tensor_shapes(normal_distributions, sample_shapes)

tfp.distributions.Normal("Standard", batch_shape=[], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1,)
Sample shape: 2
Returned sample tensor shape: (2,)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5)

tfp.distributions.Normal("Standard_Vector_Batch", batch_shape=[1], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 1)
Sample shape: 2
Returned sample tensor shape: (2, 1)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 1)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 1)

tfp.distributions.Normal("Different_Locs", batch_shape=[4], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 4)
Sample shape: 2
Returned sample tensor shape: (2, 4)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 4)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 4)

tfp.distributions.Normal("Broadc

Isto é tudo o que temos a dizer sobre `sample`: os tensores de amostra retornados têm formato `[sample_shape, batch_shape, event_shape]`.

### Computação de `log_prob` para distribuições escalares

Agora, vamos dar uma olhada em `log_prob`, que é um pouco mais complicada. `log_prob` recebe como entrada um tensor (não vazio) representando o(s) local(is) nos quais computar a `log_prob` para a distribuição. No caso mais direto e simples, esse tensor terá um formato na forma `[sample_shape, batch_shape, event_shape]`, em que `batch_shape` e `event_shape` coincidem com os formatos de lote e evento da distribuição. Lembre-se de que, para distribuições escalares, `event_shape = []`, então o tensor de entrada tem formato `[sample_shape, batch_shape]`. Neste caso, obtemos de volta um tensor de formato `[sample_shape, batch_shape]`:

In [ ]:
three_poissons = tfd.Poisson(rate=[1., 10., 100.], name='Three Poissons')
three_poissons

<tfp.distributions.Poisson 'Three_Poissons' batch_shape=[3] event_shape=[] dtype=float32>

In [ ]:
three_poissons.log_prob([[1., 10., 100.], [100., 10., 1]])  # sample_shape is [2].

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -2.0785608,   -3.2223587],
       [-364.73938  ,   -2.0785608,  -95.39484  ]], dtype=float32)>

In [ ]:
three_poissons.log_prob([[[[1., 10., 100.], [100., 10., 1.]]]])  # sample_shape is [1, 1, 2].

<tf.Tensor: shape=(1, 1, 2, 3), dtype=float32, numpy=
array([[[[  -1.       ,   -2.0785608,   -3.2223587],
         [-364.73938  ,   -2.0785608,  -95.39484  ]]]], dtype=float32)>

Observe como, no primeiro exemplo, a entrada e a saída têm formato `[2, 3]` e, no segundo exemplo, elas têm formato `[1, 1, 2, 3]`.

Isso seria tudo a ser dito se não fosse pelo broadcasting. Veja abaixo as regras quando levamos o broadcasting em conta. Descrevemos de forma geral e acrescentamos simplificações para distribuições escalares:

1. Defina `n = len(batch_shape) + len(event_shape)` (para distribuições escalares, `len(event_shape)=0`).
2. Se o tensor de entrada `t` tiver menos do que `n` dimensões, preencha seu formato adicionando dimensões de tamanho `1` à esquerda até ter exatamente `n` dimensões. Chame o tensor resultante de `t'`.
3. Faça o broadcasting das `n` dimensões na extremidade direita de `t'` em relação a `[batch_shape, event_shape]` da distribuição para a qual você está computando uma `log_prob`. Em maiores detalhes: para as dimensões em que `t'` já coincida com a distribuição, não faça nada e, para as dimensões em que `t'` tenha um singleton, replique esse singleton o número de vezes apropriado. Qualquer outra situação é um erro (para distribuições escalares, fazemos o broadcasting somente em relação a `batch_shape`, já que event_shape = `[]`).
4. Agora, finalmente podemos computar a `log_prob`. O tensor resultante terá formato `[sample_shape, batch_shape]`, em que `sample_shape` é definido como qualquer dimensão de `t` ou `t'` à esquerda das `n` dimensões na extremidade direita: `sample_shape = shape(t)[:-n]`.

Isso pode parecer confuso se você não souber o que significa, então vejamos alguns exemplos:

In [ ]:
three_poissons.log_prob([10.])

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-16.104412 ,  -2.0785608, -69.05272  ], dtype=float32)>

É feito broadcasting do tensor `[10.]` (com formato `[1]`) no `batch_shape` de 3, então avaliamos a log-probabilidade de todas as três distribuições de Poisson no valor 10.

In [ ]:
three_poissons.log_prob([[[1.], [10.]], [[100.], [1000.]]])

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[-1.0000000e+00, -7.6974149e+00, -9.5394836e+01],
        [-1.6104412e+01, -2.0785608e+00, -6.9052719e+01]],

       [[-3.6473938e+02, -1.4348087e+02, -3.2223587e+00],
        [-5.9131279e+03, -3.6195427e+03, -1.4069575e+03]]], dtype=float32)>

No exemplo acima, o tensor de entrada tem formato `[2, 2, 1]`, enquanto o objeto de distribuições tem um formato de lote igual a 3. Então, para cada uma das `[2, 2]` dimensões de amostra, o único valor fornecido sofre broadcasting em cada uma das três distribuições de Poisson.

Uma forma possivelmente útil de pensar nisso é: como `three_poissons` tem `batch_shape = [2, 3]`, uma chamada a `log_prob` deve receber um tensor cuja última dimensão seja 1 ou 3; qualquer outra coisa é um erro (as regras de broadcasting do NumPy tratam o caso especial de um escalar sendo totalmente equivalente a um tensor de formato `[1]`).

Vamos testar com a distribuição de Poisson mais complexa, com `batch_shape = [2, 3]`:

In [ ]:
poisson_2_by_3 = tfd.Poisson(
    rate=[[1., 10., 100.,], [2., 20., 200.]],
    name='Two-by-Three Poissons')

In [ ]:
poisson_2_by_3.log_prob(1.)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -17.004269 , -194.70169  ]], dtype=float32)>

In [ ]:
poisson_2_by_3.log_prob([1.])  # Exactly equivalent to above, demonstrating the scalar special case.

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -17.004269 , -194.70169  ]], dtype=float32)>

In [ ]:
poisson_2_by_3.log_prob([[1., 1., 1.], [1., 1., 1.]])  # Another way to write the same thing. No broadcasting.

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -17.004269 , -194.70169  ]], dtype=float32)>

In [ ]:
poisson_2_by_3.log_prob([[1., 10., 100.]])  # Input is [1, 3] broadcast to [2, 3].

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ -1.       ,  -2.0785608,  -3.2223587],
       [ -1.3068528,  -5.14709  , -33.90767  ]], dtype=float32)>

In [ ]:
poisson_2_by_3.log_prob([[1., 10., 100.], [1., 10., 100.]])  # Equivalent to above. No broadcasting.

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ -1.       ,  -2.0785608,  -3.2223587],
       [ -1.3068528,  -5.14709  , -33.90767  ]], dtype=float32)>

In [ ]:
poisson_2_by_3.log_prob([[1., 1., 1.], [2., 2., 2.]])  # No broadcasting.

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -14.701683 , -190.09653  ]], dtype=float32)>

In [ ]:
poisson_2_by_3.log_prob([[1.], [2.]])  # Equivalent to above. Input shape [2, 1] broadcast to [2, 3].

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -14.701683 , -190.09653  ]], dtype=float32)>

Os exemplos acima envolveram broadcasting ao longo do lote, mas o formato de amostra estava vazio. Suponhamos que tenhamos uma coleção de valores e que queiramos obter a log-probabilidade de cada valor em cada ponto no lote. Podemos fazer isso manualmente:

In [ ]:
poisson_2_by_3.log_prob([[[1., 1., 1.], [1., 1., 1.]], [[2., 2., 2.], [2., 2., 2.]]])  # Input shape [2, 2, 3].

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

Ou podemos deixar o broadcasting tratar a última dimensão de lote:

In [ ]:
poisson_2_by_3.log_prob([[[1.], [1.]], [[2.], [2.]]])  # Input shape [2, 2, 1].

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

Também podemos (talvez de uma forma menos natural) deixar o broadcasting tratar somente a primeira dimensão de lote:

In [ ]:
poisson_2_by_3.log_prob([[[1., 1., 1.]], [[2., 2., 2.]]])  # Input shape [2, 1, 3].

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

Ou podemos deixar o broadcasting tratar *as duas* dimensões de lote:

In [ ]:
poisson_2_by_3.log_prob([[[1.]], [[2.]]])  # Input shape [2, 1, 1].

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

O exemplo acima funcionou quando queríamos somente dois valores, mas suponhamos que tivéssemos uma longa lista de valores que queríamos avaliar em cada ponto do lote. Para isso, a notação abaixo, que adiciona dimensões extras de tamanho 1 à direita do formato, é bastante útil:

In [ ]:
poisson_2_by_3.log_prob(tf.constant([1., 2.])[..., tf.newaxis, tf.newaxis])

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

Esse é um caso da [notação strided slice](https://www.tensorflow.org/api_docs/cc/class/tensorflow/ops/strided-slice), que vale a pena conhecer.

Voltando a `three_poissons` para concluir, o mesmo exemplo fica assim:

In [ ]:
three_poissons.log_prob([[1.], [10.], [50.], [100.]])

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [ -16.104412 ,   -2.0785608,  -69.05272  ],
       [-149.47777  ,  -43.34851  ,  -18.219261 ],
       [-364.73938  , -143.48087  ,   -3.2223587]], dtype=float32)>

In [ ]:
three_poissons.log_prob(tf.constant([1., 10., 50., 100.])[..., tf.newaxis])  # Equivalent to above.

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [ -16.104412 ,   -2.0785608,  -69.05272  ],
       [-149.47777  ,  -43.34851  ,  -18.219261 ],
       [-364.73938  , -143.48087  ,   -3.2223587]], dtype=float32)>

## Distribuições multivariadas

Agora, vejamos as distribuições multivariadas, que têm formato de evento não vazio. Vamos avaliar as distribuições multinomiais.

In [ ]:
multinomial_distributions = [
    # Multinomial is a vector-valued distribution: if we have k classes,
    # an individual sample from the distribution has k values in it, so the
    # event_shape is `[k]`.
    tfd.Multinomial(total_count=100., probs=[.5, .4, .1],
                    name='One Multinomial'),
    tfd.Multinomial(total_count=[100., 1000.], probs=[.5, .4, .1],
                    name='Two Multinomials Same Probs'),
    tfd.Multinomial(total_count=100., probs=[[.5, .4, .1], [.1, .2, .7]],
                    name='Two Multinomials Same Counts'),
    tfd.Multinomial(total_count=[100., 1000.],
                    probs=[[.5, .4, .1], [.1, .2, .7]],
                    name='Two Multinomials Different Everything')

]

describe_distributions(multinomial_distributions)

tfp.distributions.Multinomial("One_Multinomial", batch_shape=[], event_shape=[3], dtype=float32)
tfp.distributions.Multinomial("Two_Multinomials_Same_Probs", batch_shape=[2], event_shape=[3], dtype=float32)
tfp.distributions.Multinomial("Two_Multinomials_Same_Counts", batch_shape=[2], event_shape=[3], dtype=float32)
tfp.distributions.Multinomial("Two_Multinomials_Different_Everything", batch_shape=[2], event_shape=[3], dtype=float32)


Observe como, nos três últimos exemplos, batch_shape é sempre `[2]`, mas podemos usar broadcasting para ter um `total_count` compartilhado ou `probs` compartilhadas (ou nenhum dos dois), pois, nos bastidores, é feito broadcasting deles para que tenham o mesmo formato.

A amostragem é simples e direta, dado o que já sabemos:

In [ ]:
describe_sample_tensor_shapes(multinomial_distributions, sample_shapes)

tfp.distributions.Multinomial("One_Multinomial", batch_shape=[], event_shape=[3], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 3)
Sample shape: 2
Returned sample tensor shape: (2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 3)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 3)

tfp.distributions.Multinomial("Two_Multinomials_Same_Probs", batch_shape=[2], event_shape=[3], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 2, 3)
Sample shape: 2
Returned sample tensor shape: (2, 2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 2, 3)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 2, 3)

tfp.distributions.Multinomial("Two_Multinomials_Same_Counts", batch_shape=[2], event_shape=[3], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 2, 3)
Sample shape: 2
Returned sample tensor shape: (2, 2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 2, 3)
Sample shape: [3, 4, 5]


Computar as log-probabilidades é igualmente simples e direto. Vamos trabalhar com um exemplo usando distribuições normais multivariadas diagonais (multinomiais não são otimizadas para broadcasting, já que, com as restrições de contagens e probabilidades, o broadcasting geralmente vai produzir valores inadmissíveis). Vamos usar um lote de 2 distribuições com 3 dimensões, com a mesma média, mas escalas diferentes (desvios padrão):

In [ ]:
two_multivariate_normals = tfd.MultivariateNormalDiag(loc=[1., 2., 3.], scale_diag=tf.ones([2, 3]) * [[1.], [2.]])
two_multivariate_normals

<tfp.distributions.MultivariateNormalDiag 'MultivariateNormalDiag' batch_shape=[2] event_shape=[3] dtype=float32>

Agora, vamos avaliar a log-probabilidade de cada ponto de lote em sua média e em uma média deslocada:

In [ ]:
two_multivariate_normals.log_prob([[[1., 2., 3.]], [[3., 4., 5.]]])  # Input has shape [2,1,3].

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-2.7568154, -4.836257 ],
       [-8.756816 , -6.336257 ]], dtype=float32)>

De forma equivalente exata, podemos usar a [https://www.tensorflow.org/api_docs/cc/class/tensorflow/ops/strided-slice](notação strided slice) para inserir uma dimensão extra shape=1 no meio de uma constante:

In [ ]:
two_multivariate_normals.log_prob(
    tf.constant([[1., 2., 3.], [3., 4., 5.]])[:, tf.newaxis, :])  # Equivalent to above.

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-2.7568154, -4.836257 ],
       [-8.756816 , -6.336257 ]], dtype=float32)>

Por outro lado, se não inserirmos a dimensão extra, passamos `[1., 2., 3.]` para o primeiro ponte de lote e `[3., 4., 5.]` para o segundo.

In [ ]:
two_multivariate_normals.log_prob(tf.constant([[1., 2., 3.], [3., 4., 5.]]))

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-2.7568154, -6.336257 ], dtype=float32)>

## Técnicas de manipulação de formato

### Bijetor Reshape

O bijetor `Reshape` pode ser usado para mudar o formato de *event_shape* de uma distribuição. Vejamos um exemplo:

In [ ]:
six_way_multinomial = tfd.Multinomial(total_count=1000., probs=[.3, .25, .2, .15, .08, .02])
six_way_multinomial

<tfp.distributions.Multinomial 'Multinomial' batch_shape=[] event_shape=[6] dtype=float32>

Criamos uma distribuição multinomial com um formato de evento igual a `[6]`. O bijetor Reshape permite que a tratemos como uma distribuição com formato de evento igual a `[2, 3]`.

Um `Bijector` representa uma função um para um diferenciável em um subconjunto aberto de ${\mathbb R}^n$. Os `Bijectors` são usados em conjunto com `TransformedDistribution`, que modela uma distribuição $p(y)$ em termos de uma distribuição base $p(x)$ e um `Bijector` que representa $Y = g(X)$. Vejamos na prática:

In [ ]:
transformed_multinomial = tfd.TransformedDistribution(
    distribution=six_way_multinomial,
    bijector=tfb.Reshape(event_shape_out=[2, 3]))
transformed_multinomial

<tfp.distributions.TransformedDistribution 'reshapeMultinomial' batch_shape=[] event_shape=[2, 3] dtype=float32>

In [ ]:
six_way_multinomial.log_prob([500., 100., 100., 150., 100., 50.])

<tf.Tensor: shape=(), dtype=float32, numpy=-178.21973>

In [ ]:
transformed_multinomial.log_prob([[500., 100., 100.], [150., 100., 50.]])

<tf.Tensor: shape=(), dtype=float32, numpy=-178.21973>

Essa é a *única* ação que o bijetor `Reshape` pode fazer. Ele não pode transformar dimensões de evento em dimensões de lote ou vice-versa.

### Distribuição independente

A distribuição `Independent` é usada para tratar uma coleção (ou seja, um lote) de distribuições independentes não necessariamente idênticas como uma única distribuição. De forma mais concisa, `Independent` permite converter dimensões em `batch_shape` para dimensões em `event_shape`. Vamos ilustrar com um exemplo:

In [ ]:
two_by_five_bernoulli = tfd.Bernoulli(
    probs=[[.05, .1, .15, .2, .25], [.3, .35, .4, .45, .5]],
    name="Two By Five Bernoulli")
two_by_five_bernoulli

<tfp.distributions.Bernoulli 'Two_By_Five_Bernoulli' batch_shape=[2, 5] event_shape=[] dtype=int32>

Podemos pensar nisso como um array de moedas 2x5, com as probabilidades de cara associadas. Vamos avaliar a probabilidade de um conjunto específico arbitrário de 1s e 0s:

In [ ]:
pattern = [[1., 0., 0., 1., 0.], [0., 0., 1., 1., 1.]]
two_by_five_bernoulli.log_prob(pattern)

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[-2.9957323 , -0.10536051, -0.16251892, -1.609438  , -0.2876821 ],
       [-0.35667497, -0.4307829 , -0.91629076, -0.79850775, -0.6931472 ]],
      dtype=float32)>

Podemos usar `Independent` para transformar isso em dois "conjuntos diferentes de cinco distribuições de Bernoulli", o que é útil se quisermos considerar uma "linha" de jogadas de moeda com um dado padrão como um único resultado:

In [ ]:
two_sets_of_five = tfd.Independent(
    distribution=two_by_five_bernoulli,
    reinterpreted_batch_ndims=1,
    name="Two Sets Of Five")
two_sets_of_five

<tfp.distributions.Independent 'Two_Sets_Of_Five' batch_shape=[2] event_shape=[5] dtype=int32>

Matematicamente, estamos computando a log-probabilidade de cada "conjunto" de cinco somando as log-probabilidades das cinco jogadas de moedas "independentes" no conjunto, e é daí que a distribuição recebe seu nome:

In [ ]:
two_sets_of_five.log_prob(pattern)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-5.160732 , -3.1954036], dtype=float32)>

Podemos ir além e usar `Independent` para criar uma distribuição em que eventos individuais sejam um conjunto de distribuições de Bernoulli 2x5:

In [ ]:
one_set_of_two_by_five = tfd.Independent(
    distribution=two_by_five_bernoulli, reinterpreted_batch_ndims=2,
    name="One Set Of Two By Five")
one_set_of_two_by_five.log_prob(pattern)

<tf.Tensor: shape=(), dtype=float32, numpy=-8.356134>

Vale a pena salientar que, pela perspectiva da `sample`, usar `Independent` não muda nada:

In [ ]:
describe_sample_tensor_shapes(
    [two_by_five_bernoulli,
     two_sets_of_five,
     one_set_of_two_by_five],
    [[3, 5]])

tfp.distributions.Bernoulli("Two_By_Five_Bernoulli", batch_shape=[2, 5], event_shape=[], dtype=int32)
Sample shape: [3, 5]
Returned sample tensor shape: (3, 5, 2, 5)

tfp.distributions.Independent("Two_Sets_Of_Five", batch_shape=[2], event_shape=[5], dtype=int32)
Sample shape: [3, 5]
Returned sample tensor shape: (3, 5, 2, 5)

tfp.distributions.Independent("One_Set_Of_Two_By_Five", batch_shape=[], event_shape=[2, 5], dtype=int32)
Sample shape: [3, 5]
Returned sample tensor shape: (3, 5, 2, 5)



Como um exercício para o leitor, sugerimos considerar as diferenças e similaridades entre um lote de vetores de distribuições `Normal` e uma distribuição `MultivariateNormalDiag` pela perspectiva de amostragem e log-probabilidade. Como podemos usar `Independent` para construir uma distribuição `MultivariateNormalDiag` a partir de um lote de `Normal`s? (Observe que a distribuição `MultivariateNormalDiag` não é realmente implementada desta forma).